In [1]:
import sys
sys.path.append("/home/yat-lok/workspace/tvbtools")
from tools.signaltools import SignalToolkit
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal
#plt.style.use('ggplot')
node_names = ['aCNG-L', 'aCNG-R','mCNG-L','mCNG-R','pCNG-L','pCNG-R', 'HIP-L','HIP-R','PHG-L','PHG-R','AMY-L','AMY-R', 'sTEMp-L','sTEMP-R','mTEMp-L','mTEMp-R']

In [2]:
def fig4paper_ts(caseid, grp, key, fig):
    path = '/home/yat-lok/workspace/data4project/lateralization/tvb_parameters.xlsx'
    coData = pd.read_excel(path)
    gm = np.round(coData.loc[coData['caseid'] == caseid, 'Gc'].item(), 3)
    filename = '/home/yat-lok/workspace/data4project/lateralization/LFP_critical/'+str(grp)+'/'+str(caseid)+'_'+str(gm)+'.csv'
    # filename = '/mnt/w/gc3mins/'+grp+'/'+caseid+'.h5'
    df = pd.read_csv(filename)
    
    axes = fig.add_subplot(key)
    axes.set_title(grp, loc="left")
    axes.set_yticks([])
    #axes.spines['left'].set_visible(False)
    axes.spines['top'].set_visible(False)
    axes.spines['right'].set_visible(False)
    if grp in ["MCI","AD"]:
        axes.set_xlabel("time/s")
    li = [4,5,0,1,2,3,6,7,8,9,10,11,12,13,14,15]
    for i in li:
        if grp in ['AD'] and node_names[i] in ['pCNG-L', 'pCNG-R']:
            axes.plot(np.arange(0,1,1/81920)[1000:], df.loc[:,node_names[i]][1000:],label = node_names[i], alpha=0.8)
        else:
            axes.plot(np.arange(0,1,1/81920)[1000:], df.loc[:,node_names[i]][1000:], alpha=0.8)
    plt.legend(loc="lower right")


caseids = ["4849A", "0423A", "7827A", "1477A"]
grps= ['SNC','NC','MCI','AD']
keynum = [221,222,223,224]

fig = plt.figure(figsize=(15,7), dpi=300)
for casid, grp, key in zip(caseids, grps, keynum):
    fig4paper_ts(casid, grp, key, fig)

FileNotFoundError: [Errno 2] No such file or directory: '/mnt/d/data/LFP/SNC/4849A/4849A_0.012.csv'

<Figure size 4500x2100 with 0 Axes>

In [ ]:
# def fig4paper_ts(caseid, grp, key, fig):
#     path = '/mnt/c/Users/Wayne/tvb/stat_data/Gc_Go.xlsx'
#     coData = pd.read_excel(path)
#     gm = np.round(coData.loc[coData['caseid'] == caseid, 'Gc'].item(), 3)
#     filename = '/mnt/d/data/LFP/'+str(grp)+'/'+str(caseid)+'/'+str(caseid)+'_'+str(gm)+'.csv'
#     # filename = '/mnt/w/gc3mins/'+grp+'/'+caseid+'.h5'
#     df = pd.read_csv(filename)
    
#     axes = fig.add_subplot(key)
#     axes.set_title(f"{grp}_{caseid}")
#     for i in range(16):
#         Wn = [2,10]
#         fNQ = 81920/2
#         n=2048
#         b = signal.firwin(n, Wn, nyq=fNQ, pass_zero=False, window='hamming');
#         Vlo = signal.filtfilt(b, 1, df.loc[:,node_names[i]]);   # ... and apply it to the data.
#         # b = signal.firwin(n, Wn, nyq=fNQ, pass_zero=False, window='hamming');
#         # Vhi = signal.filtfilt(b, 1, df.loc[:,node_names[i]]);   # ... and apply it to the data.
#         Vlo = SignalToolkit.hamming_filter(data = df.loc[:,node_names[i]], n=4096)
#         Vhi = SignalToolkit.hamming_filter(data=df.loc[:,node_names[i]], Wn = [10, 60], n=4096)
#         # phi = signal.angle(signal.hilbert(Vlo))  # Compute phase of low-freq signal
#         # amp = abs(signal.hilbert(Vhi))       # Compute amplitude of high-freq signal
#         if node_names[i] in ['pCNG-L', 'pCNG-R']:
#             axes.plot(np.arange(0,1,1/81920), Vlo, label=f"theta_{node_names[i]}")
#             axes.plot(np.arange(0,1,1/81920), Vhi, label=f"gamma_{node_names[i]}")
#             axes.plot(np.arange(0,1,1/81920), df.loc[:,node_names[i]], label=node_names[i], alpha=0.5)
#         # else:
#         #     axes.plot(np.arange(0,1,1/81920),df.loc[:,node_names[i]])
#     plt.legend()


# caseids = ["4849A", "0318A", "7827A", "1477A"]
# grps= ['SNC','NC','MCI','AD']
# keynum = [221,222,223,224]

# fig = plt.figure(figsize=(15,7), dpi=300)
# for casid, grp, key in zip(caseids, grps, keynum):
#     fig4paper_ts(casid, grp, key, fig)